In [2]:
%reset
import ctypes
import numpy as np
import cv2

def get_screen_size():
	user32 = ctypes.windll.user32
	screensize = (user32.GetSystemMetrics(0), user32.GetSystemMetrics(1))
	return screensize

lk_params = dict(winSize=(15, 15),
				 maxLevel=10,
				 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

feature_params = dict(maxCorners=20,
					  qualityLevel=0.3,
					  minDistance=10,
					  blockSize=7)

trajectory_len = 40 # pixel/trajectories
detect_interval = 5
trajectories = []
roi_trajectories = []
object_max_search = 0
frame_idx = 0

# Video File
vid_path = './resources/maze/with_obstacle.mp4'

# Set Video Source (Camera / Video File)
# cap = cv2.VideoCapture(0) # Camera -> use index 0 for default/builtin camera
cap = cv2.VideoCapture(vid_path) # Video File

while True:
	isFrameFound, frame = cap.read()
	
	if not isFrameFound:
		print("No Frame")
		break

	frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Convert Colored Frame to Gray Color since HOG feature/Canny needs a gray
	frame_canny = cv2.Canny(frame_gray, 100, 200) # Process Canny before plugging it to LK
	frame_lukas_kanade_tracker = frame #.copy()

	# Calculate optical flow for a sparse feature set using the iterative Lucas-Kanade Method
	if len(trajectories) > 0:
		prev_frame, current_frame = prev_gray, frame_canny
		initial_points = np.float32([trajectory[-1] for trajectory in trajectories]).reshape(-1, 1, 2) # Pixel before endpoint
		
		# calcOpticalFlowPyrLK -> optical flow-based feature tracking algorithm
		tracked_points, status, error = cv2.calcOpticalFlowPyrLK(prev_frame, current_frame, initial_points, None, **lk_params)
		end_point, status, error = cv2.calcOpticalFlowPyrLK(current_frame, prev_frame, tracked_points, None, **lk_params)
		displacement = abs(initial_points - end_point).reshape(-1, 2).max(-1)
		good = displacement < 1

		new_trajectories = []

		# Get all the trajectories
		for trajectory, (x, y), good_flag in zip(trajectories, tracked_points.reshape(-1, 2), good):
			if not good_flag:
				continue
			trajectory.append((x, y))
			if len(trajectory) > trajectory_len:
				del trajectory[0]
			new_trajectories.append(trajectory)

			cv2.circle(frame_lukas_kanade_tracker, (int(x), int(y)), 2, (0, 0, 255), -1) # Red Dots at Upper Left Frame

		trajectories = new_trajectories

		# Draw all the trajectories
		cv2.polylines(frame_lukas_kanade_tracker, [np.int32(trajectory) for trajectory in trajectories], False, (0, 255, 0))
		cv2.putText(frame_lukas_kanade_tracker, 'track counter: %d' % len(trajectories), (20, 80), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)

	# Update interval - When to update and detect new features
	if frame_idx % detect_interval == 0: # % modulo
		frame_lukas_kanade_goodfeat = np.zeros_like(frame_canny)
		frame_lukas_kanade_goodfeat[:] = 255

		# Lastest point in latest trajectory
		for x, y in [np.int32(trajectory[-1]) for trajectory in trajectories]:
			# if object_max_search < y:
			cv2.circle(frame_lukas_kanade_goodfeat, (x, y), 3, 0, -1) # Desc: Dots in the Screen, Trackpoints, Black Dots (Upper Right Frame)

		# Detect the good features to track -> Lukas-Kanade
		p = cv2.goodFeaturesToTrack(frame_canny, mask=frame_lukas_kanade_goodfeat, **feature_params)
		if p is not None:
			# If good features can be tracked - add that to the trajectories
			for x, y in np.float32(p).reshape(-1, 2):
				trajectories.append([(x, y)]) # Trackers: History of previous optical flow

	frame_idx += 1
	prev_gray = frame_canny

	# █▀ █ █ █▀█ █ █ █   █▀█ █▄ █ █   █▄█   █   █▀█ █ █ █ █▀▀ █▀█   █   █▀▀ ▄▀█ █▀▄ █ █▄ █ █▀▀   █   █ █▄ █ █▀▀ █▀
	# ▄█ █▀█ █▄█ ▀▄▀▄▀   █▄█ █ ▀█ █▄▄  █    █▄▄ █▄█ ▀▄▀▄▀ ██▄ █▀▄   █▄▄ ██▄ █▀█ █▄▀ █ █ ▀█ █▄█   █▄▄ █ █ ▀█ ██▄ ▄█
	if frame_lukas_kanade_tracker.shape[2] == 3:
		frame_lukas_kanade_tracker = cv2.cvtColor(frame_lukas_kanade_tracker, cv2.COLOR_BGR2BGRA)
	
	# █▀▀ █▀█ █▄ █ █ █ █▀▀ █▀█ ▀█▀   █ █▀▄▀█ █▀▀   ▀█▀ █▀█   █▄▄ █▀▀ █▀█ ▄▀█
	# █▄▄ █▄█ █ ▀█ ▀▄▀ ██▄ █▀▄  █    █ █ ▀ █ █▄█    █  █▄█   █▄█ █▄█ █▀▄ █▀█ to make the merging of windows possible
	frame_lukas_kanade_tracker = cv2.cvtColor(frame_lukas_kanade_tracker, cv2.COLOR_BGR2BGRA) # Lukas-Kanade
	frame_lukas_kanade_goodfeat = cv2.cvtColor(frame_lukas_kanade_goodfeat, cv2.COLOR_BGR2BGRA) # Lukas-Kanade
	frame_canny = cv2.cvtColor(frame_canny, cv2.COLOR_BGR2BGRA) # Canny

	# For No Video Frame Only
	frame_no_video = np.zeros_like(frame_canny)
	frame_no_video[:] = 100
	# print(frame_canny.shape)
	hh = int(frame_canny.shape[0] / 2)
	ww = int(frame_canny.shape[1] / 2) - (5 * 35)
	
	# █▀█ █   ▄▀█ █▀▀ █▀▀   ▀█▀ █▀▀ ▀▄▀ ▀█▀   █▀▀ █▀█ █▀█   █▀▄ █▀▀ █▀ █▀▀ █▀█ █ █▀█ ▀█▀ █ █▀█ █▄ █
	# █▀▀ █▄▄ █▀█ █▄▄ ██▄    █  ██▄ █ █  █    █▀  █▄█ █▀▄   █▄▀ ██▄ ▄█ █▄▄ █▀▄ █ █▀▀  █  █ █▄█ █ ▀█
	cv2.putText(frame_lukas_kanade_tracker, 'Using: Lukas-Kanade (flow-based feature tracking)', (20, 50), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 2)
	cv2.putText(frame_lukas_kanade_goodfeat, 'Using: Lukas-Kanade (flow estimation & good features tracking)', (20, 50), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)
	cv2.putText(frame_canny, 'Using: Canny (Edge Detection)', (20, 50), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)
	cv2.putText(frame_no_video, 'NO VIDEO', (ww, hh), cv2.FONT_HERSHEY_PLAIN, 5, (0, 255, 100), 2)
	
	# █▀▀ █▀█ █▀▄▀█ █▄▄ █ █▄ █ █▀▀   █▀▀ █▀█ ▄▀█ █▀▄▀█ █▀▀   ▀█▀ █▀█   █▀█ █▄ █ █▀▀   █ █ █ █ █▄ █ █▀▄ █▀█ █ █ █
	# █▄▄ █▄█ █ ▀ █ █▄█ █ █ ▀█ ██▄   █▀  █▀▄ █▀█ █ ▀ █ ██▄    █  █▄█   █▄█ █ ▀█ ██▄   ▀▄▀▄▀ █ █ ▀█ █▄▀ █▄█ ▀▄▀▄▀
	top_row = np.concatenate((frame_lukas_kanade_tracker, frame_lukas_kanade_goodfeat), axis=1)
	middle_row = np.concatenate((frame_canny, frame_no_video), axis=1)

	# Set the dimensions of the window you want to create
	window_width = get_screen_size()[0]
	window_height = get_screen_size()[1]

	# combined_frames = np.concatenate((top_row, middle_row, bottom_row), axis=0)
	combined_frames = np.concatenate((top_row, middle_row), axis=0)
	
	scaling_factor = min(window_height / combined_frames.shape[1], window_width / combined_frames.shape[0])
	h = window_width #int(combined_frames.shape[1] / 1.5) # if using hd camera, divide by 3 instead else 2
	w = window_height #int(combined_frames.shape[0] / 1.5)
	resized_image = cv2.resize(combined_frames, (int(combined_frames.shape[1] * scaling_factor), int(combined_frames.shape[0] * scaling_factor)))

	height, width = resized_image.shape[:2]
	
	x_position = int(abs(width - window_width) / 2)
	y_position = int(abs(height - window_height) / 2) - 30 #int(abs(height - window_height))

	# █▀ █ █ █▀█ █ █ █   █ █ █ █ █▄ █ █▀▄ █▀█ █ █ █
	# ▄█ █▀█ █▄█ ▀▄▀▄▀   ▀▄▀▄▀ █ █ ▀█ █▄▀ █▄█ ▀▄▀▄▀
	# Show Results, 
	cv2.imshow('Merged Frames', resized_image)

	windowMoved = False

	if not window_width:
		cv2.moveWindow('Merged Frames', x_position, y_position)
		windowMoved = True

	if cv2.waitKey(10) & 0xFF == ord('q'):
		break

cap.release()
cv2.destroyAllWindows()

Nothing done.


In [24]:
# import cv2 
# import numpy as np 


# # Read image 
# # image = cv2.imread('./resources/maze/with_obstacle.mp4') 
# image = cv2.imread('./resources/maze/IMG_20231226_124615.jpg') 

# # Select ROI 
# r = cv2.selectROI("select the area", image) 

# # Crop image 
# cropped_image = image[int(r[1]):int(r[1]+r[3]), 
# 					int(r[0]):int(r[0]+r[2])] 

# # Display cropped image 
# cv2.imshow("Cropped image", cropped_image) 
# cv2.waitKey(0) 
# cv2.destroyAllWindows()


import cv2
import numpy as np

def select_roi(video_path):
	# Open the video file
	cap = cv2.VideoCapture(video_path)

	# Read the first frame from the video
	ret, frame = cap.read()

	# Display the video to allow the user to select the ROI
	cv2.imshow("Select ROI", frame)
	roi = cv2.selectROI("Select ROI", frame, fromCenter=False, showCrosshair=True)

	# while True:
	# 	# Read the next frame from the video
	# 	ret, frame = cap.read()

	# 	# If the video has ended, break the loop
	# 	if not ret:
	# 		break

	# 	# Draw the rectangle on the frame
	# 	cv2.rectangle(frame, (int(roi[0]), int(roi[1])), (int(roi[0] + roi[2]), int(roi[1] + roi[3])), (0, 255, 0), 2)

	# 	# Display the frame with the selected ROI
	# 	cv2.imshow("Select ROI", frame)

	# 	# Break the loop if the user presses the 'Enter' key
	# 	if cv2.waitKey(10) & 0xFF == ord('q'):
	# 		break

	# Crop the selected ROI from the first frame
	# cropped_roi = frame[int(roi[1]):int(roi[1] + roi[3]), int(roi[0]):int(roi[0] + roi[2])]
	# cropped_roi = int(roi[1]), int(roi[1] + roi[3]), int(roi[0]), int(roi[0] + roi[2])
	cropped_roi = roi

	# Release the video capture object
	cap.release()

	# Close the window
	cv2.destroyAllWindows()

	return cropped_roi

if __name__ == "__main__":
	# Replace 'your_video_path.mp4' with the path to your video file
	# video_path = './resources/maze/with_obstacle.mp4'
	# video_path = './resources/videos/boy-walking.mp4'
	video_path = 1

	# Call the function to select and return the ROI as np.array
	roi_array = select_roi(video_path)

	# Display the cropped ROI
	# cv2.imshow("Cropped ROI", roi_array)
	print(roi_array[0])
	print(roi_array)
	cv2.waitKey(0)
	cv2.destroyAllWindows()


16
(16, 157, 86, 238)
